In [16]:
import dgl
import torch

def get_subgraph(graph, indices):
    k = 2
    # 提取入向k跳子图
    subgraph_in, _ = dgl.khop_in_subgraph(graph, indices, k)

    # 提取出向k跳子图
    subgraph_out, _ = dgl.khop_out_subgraph(graph, indices, k)

    # 合并两个子图的节点
    nodes_union = torch.unique(torch.cat([subgraph_in.nodes(), subgraph_out.nodes()], dim=0))
    subgraph_union = graph.subgraph(nodes_union)

    # 在子图中标记原始图的节点索引
    subgraph_union.ndata['original_indices'] = nodes_union

    return subgraph_union

# 创建一个示例图
src = torch.tensor([0, 1, 2, 3, 1, 4, 5, 4, 5, 2, 0, 6, 6, 7, 7, 8, 8, 9])
dst = torch.tensor([1, 2, 3, 4, 4, 5, 4, 5, 2, 0, 6, 7, 8, 9, 6, 7, 9, 6])
g = dgl.graph((src, dst))

# 测试代码
indices = torch.tensor([1, 4])  # 选择节点 1 和 4
subgraph_union = get_subgraph(g, indices)
print("合并子图的节点:", subgraph_union.nodes())
print("合并子图的边:", subgraph_union.edges())
print("原始图的节点索引:", subgraph_union.ndata['original_indices'])


合并子图的节点: tensor([0, 1, 2, 3, 4, 5])
合并子图的边: (tensor([0, 1, 1, 2, 2, 3, 4, 4, 5, 5]), tensor([1, 2, 4, 3, 0, 4, 5, 5, 4, 2]))
原始图的节点索引: tensor([0, 1, 2, 3, 4, 5])


In [17]:
import torch
import dgl 
num_nodes, num_edges = 100, 200
src_nodes = torch.randint(0, num_nodes, (num_edges,))
dst_nodes = torch.randint(0, num_nodes, (num_edges,))
graph = dgl.graph((src_nodes, dst_nodes))
graph = dgl.add_self_loop(graph)
graph

Graph(num_nodes=100, num_edges=300,
      ndata_schemes={}
      edata_schemes={})

In [18]:
graph.num_nodes()

100

In [19]:
graph.ndata['features'] = torch.randn(graph.number_of_nodes(), 128)
graph

Graph(num_nodes=100, num_edges=300,
      ndata_schemes={'features': Scheme(shape=(128,), dtype=torch.float32)}
      edata_schemes={})

In [20]:
graph.nodes()

tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
        36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53,
        54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71,
        72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89,
        90, 91, 92, 93, 94, 95, 96, 97, 98, 99])

In [21]:
dgl.node_subgraph(graph, [1, 3, 5])

Graph(num_nodes=3, num_edges=3,
      ndata_schemes={'features': Scheme(shape=(128,), dtype=torch.float32), '_ID': Scheme(shape=(), dtype=torch.int64)}
      edata_schemes={'_ID': Scheme(shape=(), dtype=torch.int64)})

In [22]:
print(dgl.node_subgraph(graph, [1, 3, 5]).edges())

(tensor([0, 1, 2]), tensor([0, 1, 2]))


In [23]:
dgl.bfs_nodes_generator(graph, [0])

(tensor([0]), tensor([21, 11]))

In [24]:
dgl.bfs_nodes_generator(graph, [1])

(tensor([1]),)

In [25]:
dgl.readout_nodes(graph, 'features', op="mean").shape

torch.Size([1, 128])

In [26]:
dgl.khop_graph(graph, 1)

Graph(num_nodes=100, num_edges=300,
      ndata_schemes={'features': Scheme(shape=(128,), dtype=torch.float32)}
      edata_schemes={})

In [27]:
newgraph, _ = dgl.khop_in_subgraph(graph, [99], 2)
newgraph.nodes()

tensor([0, 1, 2, 3, 4])

In [34]:
new_idx = newgraph.ndata['_ID']
new_idx

tensor([15, 26, 28, 70, 99])

In [29]:
new_idx = newgraph.nodes()

tensor([0, 1, 2, 3, 4])

In [39]:
index_map = {int(original): i for i, original in enumerate(newgraph.ndata['_ID'].tolist())}
index_map

{15: 0, 26: 1, 28: 2, 70: 3, 99: 4}